# Refleksja w Rubim


## Struktura dostęp do informacji o typie, polach i metodach
## Dynamiczne ładowanie klas
    
<br/>

## dr inż. Aleksander Smywiński-Pohl

## apohllo@agh.edu.pl

## http://apohllo.pl/dydaktyka/programowanie-obiektowe

## konsultacje: czwartek 14:45 - 15:45, pokój 4.61



<img src="img/ruby-horizontal.svg" width="800"/>

# Pomoc wbudowana w REPLa

In [ ]:
[1] pry(main)> require 'dry/types'
=> true
[2] pry(main)> ls Dry
constants: Configurable  Container  Core  Equalizer  Logic  Types
Dry.methods: Equalizer
[3] pry(main)> ls Dry::Equalizer
constants: Methods
Object.methods: yaml_tag
Module.methods: constants  nesting  used_modules
[4] pry(main)> ? Dry::Equalizer#constants

From: variable.c (C Method):
Owner: Module
Visibility: public
Signature: constants(*arg1)
Number of lines: 12

Returns an array of the names of the constants accessible in
mod. This includes the names of constants in any included
modules (example at start of section), unless the inherit
parameter is set to false.

The implementation makes no guarantees about the order in which the
constants are yielded.

  IO.constants.include?(:SYNC)        #=> true
  IO.constants(false).include?(:SYNC) #=> false

Also see Module::const_defined?.


# Informacja o typie

In [2]:
puts 1.class()
puts "Ala ma kota".class()
puts nil.class()

Integer
String
NilClass


# Weryfikacja przynależności do typu

In [3]:
object = "Ala ma kota"

if(object.is_a?(String))
  puts "Obiekt jest łańcuchem znaków"
end

Obiekt jest łańcuchem znaków


In [4]:
object = "Ala ma kota"

if(String === object)
    puts "Obiekt jest łańcuchem znaków"
end

Obiekt jest łańcuchem znaków


In [5]:
object = [1,2,3]

case(object)
when String
  puts "Obiekt jest łańuchem znaków"
when Array
  puts "Obiekt jest tablicą"
end

Obiekt jest tablicą


# Informacje o hierarchii klas

In [17]:
class Animal
end

class Mammal < Animal
end

class Cat < Mammal
end

class Bat < Mammal
end

p Cat.new
p Cat.new.class
p Cat.new.class.superclass
p Cat.new.class.class.class
p Cat.new.class.class.class.superclass.superclass.superclass.superclass

#<Cat:0x000055dcc3cf2038>
Cat
Mammal
Class
nil


In [18]:
def ancestors(klass, chain=[])
  klass.superclass ? ancestors(klass.superclass, [klass] + chain) : [klass] + chain
end

puts ancestors(Bat)

[BasicObject, Object, Animal, Mammal, Bat]


In [19]:
class Whale < Mammal
end

class Reptile < Animal
end

class Snake < Reptile
end

In [20]:
[1,2,3].zip(["a", "b", "c"]).to_a

[[1, "a"], [2, "b"], [3, "c"]]

In [23]:
[1,2,3,4,5,6].each_cons(2).to_a

[[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]]

In [24]:
[1,2,3,4,5].
  zip(["a", "b", "c", "d", "e"]).
  each_cons(2).
  each{|(a1, b1), (a2, b2)| puts "#{a1} - #{b1} : #{a2} - #{b2}"}

1 - a : 2 - b
2 - b : 3 - c
3 - c : 4 - d
4 - d : 5 - e


In [25]:
def commonAncestor(klass1, klass2)
  ancestors(klass1).zip(ancestors(klass2)).each_cons(2) do |(grandparent1, grandparent2), (parent1, parent2)|
    return grandparent1 if parent1 != parent2 
  end
end

:commonAncestor

In [26]:
puts commonAncestor(Bat, Whale)
puts commonAncestor(Snake, Bat)
puts commonAncestor(Snake, Reptile)
puts commonAncestor(1.class, Bat)

Mammal
Animal
Reptile
Object


# Dynamiczne ładowanie klas

In [28]:
class Car
  def go
    puts "Brum brum"
  end
end

class Airplane
  def go
    puts "Bzzzzz"
  end
end

class Rocket
  def go
    puts "BUUUUUUM"
  end
end

:go

In [29]:
["Car", "Car", "Rocket", "Airplane"].each do |classname|
  Class.const_get(classname).new.go
end

Brum brum
Brum brum
BUUUUUUM
Bzzzzz


["Car", "Car", "Rocket", "Airplane"]

# Refleksja

In [30]:
puts Car.new.public_methods.sort

[:!, :!=, :!~, :<=>, :==, :===, :=~, :__binding__, :__id__, :__send__, :class, :clone, :define_singleton_method, :display, :dup, :enum_for, :eql?, :equal?, :extend, :freeze, :frozen?, :go, :hash, :inspect, :instance_eval, :instance_exec, :instance_of?, :instance_variable_defined?, :instance_variable_get, :instance_variable_set, :instance_variables, :is_a?, :itself, :kind_of?, :method, :methods, :nil?, :object_id, :pretty_inspect, :pretty_print, :pretty_print_cycle, :pretty_print_inspect, :pretty_print_instance_variables, :private_methods, :protected_methods, :pry, :public_method, :public_methods, :public_send, :remove_instance_variable, :respond_to?, :send, :singleton_class, :singleton_method, :singleton_methods, :taint, :tainted?, :tap, :then, :to_enum, :to_json, :to_s, :trust, :untaint, :untrust, :untrusted?, :yield_self]


In [31]:
puts (Car.new.public_methods - 1.public_methods).sort

[:go]


In [43]:
class Bike
  #def initialize
  #  @speed = 0
  #end

  def go
    @speed = 10
  end

  def stop
    @speed = 0
  end

  def riding?
    @speed > 0
  end
  
  def speed
    @speed
  end
end

:speed

In [44]:
puts Bike.new.instance_variables
bike = Bike.new
p bike.speed
bike.go
bike.instance_variables

[:@speed]
0


[:@speed]

# Dynamiczne wywoływanie metod

In [45]:
bike = Bike.new

puts bike.public_send(:riding?)
bike.public_send("go")
puts bike.public_send("riding?")

false
true


In [47]:
bike = Bike.new
#bike.public_send("initialize")
bike.send("initialize")

0

# Dynamiczny dostęp do pól

In [48]:
bike = Bike.new
puts bike.instance_variable_get("@speed")
bike.instance_variable_set("@speed", 10)
puts bike.instance_variable_get("@speed")
puts bike.speed

0
10
10


In [49]:
class Bike
  def instance_variable_set(name, value)
    raise "Don't do it!"
  end
end

:instance_variable_set

In [ ]:
bike = Bike.new
puts bike.instance_variable_get("@speed")
bike.instance_variable_set("@speed", 10)
puts bike.instance_variable_get("@speed")

0


RuntimeError: Don't do it!

# Dynamiczne proxy

In [51]:
class Wrapper
  def initialize(object)
    @object = object
    @invocations = []
  end

  def method_missing(*arguments)
    @invocations << arguments 
    @object.public_send(*arguments)
  end

  def replay
    @invocations.each do |arguments|
      @object.public_send(*arguments)
    end
  end
end

:replay

In [55]:
wrapper = Wrapper.new([])
wrapper << 1
wrapper << 2
wrapper << 3
puts wrapper.to_a.join(", ")

wrapper.replay

puts wrapper.to_a.join(", ")

wrapper.replay

puts wrapper.to_a.join(", ")

1, 2, 3
1, 2, 3, 1, 2, 3
1, 2, 3, 1, 2, 3, 1, 2, 3


In [56]:
wrapper = Wrapper.new({})
wrapper[:a] = 1
wrapper[:b] = 2

puts wrapper.to_h

wrapper.replay

puts wrapper.to_h

{:a=>1, :b=>2}
{:a=>1, :b=>2}


![Pytania? ](img/question.jpg)